# Function Setup

## 5.) Clustering related methods

In [43]:
# Get the number of members for each cluster
def kmeans_counts(data_labels, clusters =100):
    
    counts = np.zeros(shape = (clusters,1))
    for i in range(clusters):
        counts[i] = (data_labels == i).sum()
    
    return counts

## Method 1: Use labeling of k-Means Clustering and optimize the corresponding representative

In [10]:
## This Procedure combines the functions 'get_representatives_model', 'get_representative_configuration' and
# 'get_representatives_train', which operate all on a single model basis
# Now we combine the procedures for the list of models for all clusters
# Automize Procedure to obtain representatives of pre-determined clusters
# Input: List of data for given number of clusters.
# Optional input: loss_type, patience of early stopping (es_patience), model name (for saving the best model), epochs
#                optimizer, learning rate of optimizer, decay of optimizer
# Output: list of representatives and list of models

def cluster_ann(y_lst, model_pretrained, N_ensembles =5, qualitative_option = False, 
                optimizer = 'adam', loss_type = 'mse', metric_type = 'mae',
                N_epochs = 4000, N_data = 100, es_patience = 15, 
                option_pretrain = False, optimizer_pretrain = 'adam',
                option_callbacks = 'all',
        wd_cluster=r'C:\Users\mark.kiermayer\Documents\Python Scripts\Master Thesis - Code\checkpoints\Cluster'):
    
    t_start = time.time()
    # Parameters
    N_clusters = len(y_lst)
    N_features = 4
    representatives = np.zeros(shape=(N_clusters,N_features))
    representatives_pv = np.zeros(shape=(N_clusters,y_lst[0].shape[1]))
    model_weights = []
    history = []
    times = np.zeros(N_clusters)
    
    # General Config
    constraint_opt = False
    midlayer = False
    
    # Get general model (for all clusters)
    model_clustering = get_representatives_model(N_input=N_features, scale=V_max, N_ensembles=N_ensembles, 
                                                   mid_layer_option = midlayer,
                                                   constraint_option=constraint_opt, 
                                                   qualitative_option = qualitative_option, version = 'v2')
    
    #Get initial weights, to reset them for every cluster
    # Only the weights between input layer and 1st hidden layer are trainable/ will change thru-out training
    w_clustering_init = model_clustering.layers[1].get_weights()
    
    # Import weights for model (Note: Including a constraint influences the number of layers)
    get_representative_configuration(model_update = model_clustering, model_weights= model_pretrained, 
                                         layer_start = 3)  
    
    
    # Function to obtain representative contracts (i.e. hidden output)
    get_cluster_representative = K.function(inputs = [model_clustering.layers[0].input],
                                  outputs=[model_clustering.layers[1].output])
    
    print('Model set up. Time required: ' + str(np.round_(time.time()-t_start, decimals = 2))+ ' sec.')
    
    for i in range(N_clusters):
        if i>0:
            # reset (trainable) weights
            model_clustering.layers[1].set_weights(w_clustering_init)
            
        y = y_lst[i]
        t_start = time.time()
        print('Model for Cluster {} of {}'.format(i+1,len(y_lst)))        
        print('\t Training in progress')
        
        if option_pretrain:
            # Optional Pretraining (e.g. with Adam optimizer)
            get_representatives_train(model = model_clustering, x = None, y = y, 
                                      optimizer= optimizer_pretrain, loss = loss_type, metric = metric_type,
                                      N_epochs = N_epochs, es_patience = es_patience, cluster_number = str(i), 
                                      N_x = N_data, option_callbacks = option_callbacks,
                                      wd_cluster=wd_cluster)
        
        # Train model (e.g. mit AdaDelta optimizer)
        hist = get_representatives_train(model = model_clustering, x = None, y = y, 
                                  optimizer= optimizer, loss = loss_type, metric = metric_type,
                                  N_epochs = N_epochs, es_patience = es_patience, cluster_number = str(i), 
                                  N_x = N_data,wd_cluster=wd_cluster)
        history.append(hist.history)
        times[i] = np.round_(time.time()-t_start, decimals = 2)
        print(' \t Cluster {} completed. Time passed '.format(i+1)+ str(times[i])+ ' sec.')
        
        # Save weights of trainable layer -> model can be reconstructed later on
        model_weights.append(model_clustering.layers[1].get_weights())
        
        # Save representative (i.e. hidden output)
        representatives[i,:] = get_cluster_representative([np.array([1,1,1,1]).reshape((1,4))])[0]
        # Save corresponding reserve
        representatives_pv[i,:] = model_clustering.predict(x = np.array([1,1,1,1]).reshape((1,4)))[0]
    
    return [representatives, representatives_pv, model_weights, times, history]

In [5]:
# Create Model for the determination of a cluster's representative
# Procedure will have to be repeated for each cluster
# Input: N number of members of cluster, scale: scaling factor in Ensemble model
# Output: Model
# Options: mid_layer_option: include layer between flattened input and dense layer with 
#          _features units, to bundle information
#         contraint_option: Include unit_constraint (sum of exiting weights for all neuron <1) in all layers
#         qualitative_option: Include qualitative model

def get_representatives_model(N_input=None, N_features = 4, scale=1,  version = 'v1', N_repeat = 41, N_output = 41, 
                              N_ensembles = 1, constraint_option = False, mid_layer_option = True,
                              qualitative_option = False):
    
    count = 0
    if version == 'v1':
        # Version v1: Use all members of (by kMeans) predefined cluster to create a representative contract
        INPUT = Input(shape = (N_input,N_features))
        count +=1
        x = Flatten()(INPUT)
        count +=1
        if mid_layer_option:
            if constraint_option:
                x = Dense(units = max(int((N*N_features)/10), 4), activation = 'linear', 
                          kernel_constraint =keras.constraints.MinMaxNorm(min_value=-1.0,max_value=1.0,rate=0.8,axis=0),
                          #keras.constraints.UnitNorm(axis=0), 
                          bias_constraint = keras.constraints.MinMaxNorm(min_value=-1.0, max_value=1.0, rate=.8, axis=0))(x)
                          #keras.constraints.UnitNorm(axis=0))
                count +=1          
                x = Activation(activation='tanh')(x)
                count +=1
            else:
                x = Dense(units = max(int((N*N_features)/10), 4), activation = 'tanh')(x)
                count +=1
                
        if constraint_option:
            x = Dense(units = N_features, activation = 'linear', 
                      kernel_constraint = keras.constraints.MinMaxNorm(min_value=-1.0,max_value=1.0,rate=0.8,axis=0),
                      #keras.constraints.UnitNorm(axis=0), 
                      bias_constraint = keras.constraints.MinMaxNorm(min_value=-1.0,max_value=1.0,rate=0.8,axis=0))(x)
                      #keras.constraints.UnitNorm(axis=0))
            count +=1
            x = Activation(activation = 'tanh')(x)
            count +=1
        else:
            x = Dense(units = N_features, activation = 'tanh')(x)
            count +=1
            
    elif version == 'v2':
        # Version v2: Only the cummulative reserve of the cluster is relevant for the model, not the members' features
        # Model will find a representative contract to optimally fit the target (with no respect to the input contracts)
        INPUT = Input(shape = (N_features,))
        count +=1
        x = Dense(units = N_features, activation = 'tanh')(INPUT)
        count +=1
        

    else:
        print('Version of model unknown!')
        pass
    
    # Proceed with Agglomerated information (regardless whether model version v1 or v2) 
    # and use non-trainable pretrained model
    
    # Note: This is contrary to the previous data preparation
    # Here: No default value input when contract matured
    x = RepeatVector(n = N_repeat)(x) 
    count +=1
    #x = Lambda(lambda x_val: data_full_transform_test(x_val, n_aim = N_output, input_type = 'scaled', Max_min = Max_min))

    if N_ensembles >1:
        # include previous choice of Ensemble Model
        OUTPUT = combine_models(input_layer = x, n_ensembles =N_ensembles,
                                model_qualitative_option = qualitative_option, 
                                model_qualitative_type = 'plain',
                               load_weights = False, weights_ensembles = None, 
                               weights_qualitative = None, scale = scale, LSTM_nodes = [N_output], 
                                output_nodes = N_output,
                                  final_dense_layer = True, dense_act_fct = 'tanh', act_fct_special = False, 
                                return_option = 'output')
        
    else: # For a 1-Model-Ensemble the average-Layer cannot be evaluated
        OUTPUT = create_rnn_model(model_input=x, nodes=[N_output],  n_output=N_output, final_dense_layer = True, 
                     dense_act_fct = 'tanh', act_fct_special = False,
                     optimizer_type='adam', loss_type='mse', metric_type='mae', dropout_option=False, 
                     dropout_share=[0.2,0.2], lambda_layer = True, lambda_scale =scale, log_scale=True, 
                     model_compile = True, return_option = 'output', branch_name = '', input_type = '3D')
        
        
        
    model = Model(inputs = INPUT, outputs = OUTPUT)

    for i in range(count,len(model.layers)):
        model.layers[i].trainable = False
    
    return model

In [21]:
# Transfer pretrained weights to model and set corresponding parts as non-trainable
# Input: weights
def get_representative_configuration(model_update, model_weights, layer_start = 5, optimizer = 'adam', 
                                     loss = 'mse', metric = 'mae'):
    
    n = len(model_weights.layers)
    # Check if model suitable
    if(len(model_update.layers) != n+layer_start-1):
        print('Inputs are not compatible!')
        return
    
    # Compile model
    # Note: Every time you compile, the weights are reset to default, random initialization
    model_update.compile(optimizer = optimizer, loss = loss, metrics = [metric] )
       
    # If model suitable, continue with layer-wise weights-transfer
    for i in range(n-1):
        # For model to update: skip 4 Layers to get to Ensemble Part
        # For Ensemble Model: skip 1 Layer (i.e. Input Layer)
        model_update.layers[layer_start+i].set_weights(model_weights.layers[1+i].get_weights())
        
    return

In [17]:
## Train a single model of the list of clustering models
def get_representatives_train(model,y,x= None, N_epochs = 2000, optimizer = 'adam', loss = 'mse', metric = 'mae',
                              es_patience = 500, cluster_number = None, show_progress = False,
                              option_callbacks = 'es', N_x = 1,
        wd_cluster = r'C:\Users\mark.kiermayer\Documents\Python Scripts\Master Thesis - Code\checkpoints\Cluster'):
    
    model.compile(optimizer = optimizer, loss = loss, metrics = [metric])
    
    if x == None:
        # Create input (1,1,1,1) and repeat targets accordingly
        x = np.tile(A = [1,1,1,1], reps = N_x).reshape((N_x, 4))
        y = np.tile(y, reps = N_x).reshape((N_x, y.shape[1]))
    
    if (option_callbacks == 'all') |(option_callbacks == 'es')|(option_callbacks == 'mc'):
        if (option_callbacks == 'all')|(option_callbacks == 'es'):
            # patient early stopping
            es = EarlyStopping(monitor='loss', mode='min', verbose=show_progress, patience=es_patience)
        if (option_callbacks == 'all')|(option_callbacks == 'mc'):
            # save only best configuration
            mc = ModelCheckpoint(wd_cluster+r'\best_model_cluster_{}.h5'.format(cluster_number), monitor='loss', 
                                 mode='min', verbose=show_progress, save_best_only=True)
        # Fit model
        if (option_callbacks == 'mc'):
            hist = model.fit(x,y,batch_size=1, epochs = N_epochs, callbacks= [mc], verbose = show_progress)
        elif (option_callbacks == 'es'):
            hist = model.fit(x,y,batch_size=1, epochs = N_epochs, callbacks= [es], verbose = show_progress)
        else:
            hist = model.fit(x,y,batch_size=1, epochs = N_epochs, callbacks= [es,mc], verbose = show_progress)     
    else:
        hist = model.fit(x,y,batch_size=1, epochs = N_epochs, verbose = show_progress)  
    
    return hist

### Method 2: Utilize concept of self organizing feature maps to obtain (alternative) clustering
### (Not fruitful so far)

In [49]:
## Create Competitive Clustering ANN, 
## with integrated reserve-prediction-ensemble model (pretrained and non-trainable) and supervised by reserve profile
## Option 1: Output Features (Age,Sum ins, Duration, Age of Contract)
## Option 2: Output Reserve, i.e. Use Model from option 1 and add the model for predicting reserves as configured in Part I
def create_clustering_competitive(INPUT, N_clusters, PV_max = 1, N_features = 4, N_timesteps_ensemble = 41, 
                                  N_competitive_repeat = 20, dropout_option = True, output_type = 'reserve'):
    
    #
    if (output_type != 'reserve') & (output_type != 'feature'):
        print('unknown output_type.')
        return
    
    x = Dense(units=N_clusters, use_bias= True, activation = 'linear')(INPUT)
    if droptout_option:
        x = Dropout(rate = 0.2)(x)
    x = Dense(units=N_clusters, activation = 'relu')(x)
    if droptout_option:
        x = Dropout(rate = 0.2)(x)
    
    # Version 1: Use Lambda-Layer to implement Competitive Layer 
    # Important: Set Layer as non-trainable lateron
    #x = Lambda(lambda x_val: np.eye(N_clusters, -K.argmax(x, axis = 0)))(x)
    #np.eye(N = N_clusters, M = None, k = -K.argmax(x_val, axis = 0)))(x)
    
    # Version 2: implement competitive layer using RNN
    x = RepeatVector(N_competitive_repeat, name = 'RNN_Data_Prep_0')(x)
    x = SimpleRNN(units = N_clusters, use_bias=False, name = 'Competitive_Cluster_Layer', activation = 'relu')(x)
    x = Activation(activation='softmax')(x)
    
    # ensure that each node of this output is in [-1,+1] -> cluster representative features are explicitely given!
    x = Dense(units = N_features, activation = 'tanh', name = 'Representative_Features')(x) 
    
    if output_type == 'feature':
        model = Model(inputs = INPUT, outputs = x)
        return model
    else:
    
        x = RepeatVector(N_timesteps_ensemble, name = 'RNN_Data_Prep_1')(x)

        #x = combine_models_test(x, n_ensembles =1, model_qualitative_option = False, 
        #                   input_layer_qualitative = None, load_weights = False, weights_ensembles = None, 
        #                   weights_qualitative = None, scale = PV_max, LSTM_nodes = [41], output_nodes = 41,
        #                   final_dense_layer = True, dense_act_fct = 'tanh', act_fct_special = False, return_option = 'output')

        # x value will be used for quantative part to -> use y
        y = create_multiple_rnn_models(5, x,nodes=[41],  n_output=41, final_dense_layer = True, 
                                   dense_act_fct = 'tanh', optimizer_type='adam', loss_type='mse', 
                                   metric_type='mae', dropout_option=False, 
                                   lambda_layer = True, lambda_scale =PV_max, log_scale=False, model_compile = True, 
                                   return_option = 'output')
        y = Average(name = 'Create_Ensemble')(y)

        z = Lambda(lambda x_val: x_val[:,:,2:4], name = 'Extract_Info')(x)

        z = create_model_qualitative(z, return_option = 'output')
        z = Lambda(lambda x_val: tf.round(x_val), name = 'Binary_Transformation')(z)
        OUTPUT = multiply([y,z], name = 'Combine_Models')
        model = Model(inputs = INPUT, outputs = OUTPUT)

        return model

In [ ]:
## Transfer the pre-trained weight configuration of the ensemble model 
# to the ensemble model included as a non-trainable part of the clustering model
# Note: Clustering model has  layers previous to passing on the information to the ensemble
# For model_ensemble we have to skip the (nonexisting) input-layer's weights
def cluster_model_transfer_weights(cluster_model, model_ensemble, N_layer_start = 10, N_layer_compete = 6, 
                                   n_clusters = 100, epsilon = 0.1, ensemble_option = True):
    
    # Check for comformity
    if (len(cluster_model.layers) != len(model_ensemble.layers)+N_layer_start-1)&ensemble_option:
        print('Models have varying layer configurations and are not compatible for this method.')
        return
    
    if ensemble_option:
        # Import weights of Ensemble Model
        for i in range(len(model_ensemble.layers)-1):
            cluster_model.layers[N_layer_start+i].set_weights(model_ensemble.layers[i+1].get_weights())
            # Pretrained Ensemble Section non-trainable
            cluster_model.layers[N_layer_start+i].trainable = False
        
    # Set weights for Competitive RNN Model
    # Set 1: Pass on Information from previous layer (without changes)
    W_pass = np.eye(N=n_clusters,M= n_clusters)
    # Set 2: Competing Weights (i.e. effect of previous value)
    W_compete = np.full(shape = (n_clusters,n_clusters), fill_value= -epsilon)
    np.fill_diagonal(W_compete,val = 1)
    cluster_model.layers[N_layer_compete].set_weights([W_pass, W_compete])
    # Competitive Layer non-trainable
    cluster_model.layers[N_layer_compete].trainable = False
    
    
    return

## Evaluate Results

In [61]:
## function which compares (or visualizes) agglomeration
## We want to compare the baseline kMeans with the ANN Agglomeration
## For the ANN we have to consider the predicted reserve (by part I), which
# is integrated in the ANN-Model for clustering
# and compare this to the reserve based on classical calculation and the representatives
# obtain by the ANN-Cluster-Model
# option_plot_selection: insert a a list of numbers, representing the selcted clusters to be plotted

def analyze_agglomeration(baseline, y, Max_min, interest_rate, x = None, include_ann = False,
                          ann_representatives = None, ann_prediction = None, 
                          option = 'plot', individual_clusters = False, option_cluster_fit = True,
                          n_columns = 5, figsize = (20,30), option_plot_selection = None):
    
    #Number of clusters
    n_cl = baseline.cluster_centers_.shape[0]
    # Number of members per cluster
    count = kmeans_counts(baseline.labels_,n_cl)
    # size of output (vector of policy values)
    n_out = targets.shape[1]
    
    # Round re-Transformed representatives (duration and aoc needs to be integer valued!!)
    # Baseline
    approx_low = np.floor(data_re_transform_features(baseline.cluster_centers_, Max_min, 
                                                     option= 'conditional')).astype('int')
    approx_up = np.ceil(data_re_transform_features(baseline.cluster_centers_, Max_min, 
                                                   option= 'conditional')).astype('int')
    if include_ann == True:
        ann_approx_up = np.ceil(data_re_transform_features(ann_representatives, 
                                                        Max_min, option= 'conditional')).astype('int')
        ann_approx_low = np.floor(data_re_transform_features(ann_representatives, 
                                                        Max_min, option= 'conditional')).astype('int')
    
    # Calculate targets for (floored and ceiled) centroids for baseline and
    # optionally: also for representatives obtained by ANN
    PV_low = np.zeros(shape = (n_cl, n_out))
    PV_up = np.zeros(shape = (n_cl, n_out))
    ann_PV_up = np.zeros(shape = (n_cl, n_out))
    ann_PV_low = np.zeros(shape = (n_cl, n_out))
    for i in range(n_cl):
        
        PV_low[i,0:max(approx_low[i,2]-approx_low[i,3],0)+1] = get_termlife_reserve_profile(
                                                                age_curr = approx_low[i,0], 
                                                                Sum_ins= approx_low[i,1],
                                                                 duration = approx_low[i,2], 
                                                                interest = interest_rate, 
                                                                 age_of_contract = approx_low[i,3], 
                                                                  option_past = False)
        
        PV_up[i,0:max(approx_up[i,2]-approx_up[i,3],0)+1] = get_termlife_reserve_profile(
                                                                age_curr = approx_up[i,0], 
                                                                Sum_ins= approx_up[i,1],
                                                                 duration = approx_up[i,2], 
                                                                interest = interest_rate, 
                                                                 age_of_contract = approx_up[i,3], 
                                                                  option_past = False)
        # Optional: ALso for representatives of ANN
        if include_ann == True:
            dur_up = ann_approx_up[i,2]
            aoc_up = ann_approx_up[i,3]
            dur_low = ann_approx_low[i,2]
            aoc_low = ann_approx_low[i,3]
            ann_PV_up[i,0:dur_up-aoc_up+1] = get_termlife_reserve_profile(age_curr=ann_approx_up[i,0], 
                                                                 Sum_ins= ann_approx_up[i,1], 
                                                                 duration=dur_up, 
                                                                 interest= interest_rate, 
                                                                 age_of_contract= aoc_up,option_past= False)
            ann_PV_low[i,0:dur_low-aoc_low+1] = get_termlife_reserve_profile(age_curr=ann_approx_low[i,0], 
                                                                 Sum_ins= ann_approx_low[i,1], 
                                                                 duration=dur_low, 
                                                                 interest= interest_rate, 
                                                                 age_of_contract= aoc_low,option_past= False)    


    # Calculate actual targets per cluster 
    targets_cl = np.zeros(shape = (n_cl, n_output))
    for i in range(n_cl):
        index = baseline.labels_ == i
        targets_cl[i,:] = targets[index,:].sum(axis=0)/count[i]

 
    
    if option == 'plot':
        
        if individual_clusters == True:
            
            
            if option_plot_selection == None: # Plot all clusters C_1,..,C_K
                fig, ax = plt.subplots(nrows = np.ceil(n_cl/n_columns).astype('int'), 
                                       ncols = n_columns, figsize = figsize)
                ax = ax.flatten()

                for i in range(n_cl):
                    # Actual Targets
                    ax[i].plot(targets_cl[i,:], 'r*', label = '$R(P)$')
                    # Reserve based on K-Means clustering
                    ax[i].plot(PV_up[i,:], linestyle = ':', color = 'grey', 
                               label = r'$ R( \lceil\tilde{P}_0\rceil), R( \lfloor\tilde{P}_0\rfloor)  $')
                    ax[i].plot(PV_low[i,:], linestyle = ':', color = 'grey')

                    if i%n_columns==0: # first column
                        ax[i].set_ylabel('Policy Value', fontsize = 'large')
                    if i>= (n_columns*(np.ceil(n_cl/n_columns).astype('int')-1)): # last row
                        ax[i].set_xlabel('Time, $t$', fontsize = 'large')

                    if include_ann == True:
                        # Predicted Reserve by ANN
                        ax[i].plot(ann_prediction[i,:], color = 'blue', label = r'$\hat{f}_\mathcal{N}(\tilde{P}_{\mathcal{N}})$')
                        # Reserve based on classical calculation using representative contracts of ANN approach
                        ax[i].plot(ann_PV_up[i,:], color = 'orange', 
                                   label = r'$ R( \lceil\tilde{P}_\mathcal{N}\rceil), R( \lfloor\tilde{P}_\mathcal{N}\rfloor)  $')
                        ax[i].plot(ann_PV_low[i,:], color = 'orange') 

                    if i == 0:
                        ax[i].legend()

                plt.tight_layout()
            else: ### Plot only selection of clusters C_1,...,C_[option_plot_selection]
                fig, ax = plt.subplots(nrows = np.ceil(len(option_plot_selection)/n_columns).astype('int'), 
                                       ncols = n_columns, figsize = figsize)
                ax = ax.flatten()

                for i in range(len(option_plot_selection)):
                    # Actual Targets
                    ax[i].plot(targets_cl[option_plot_selection[i],:], 'r*', label = '$R(P)$')
                    # Reserve based on K-Means clustering
                    ax[i].plot(PV_up[option_plot_selection[i],:], linestyle = ':', color = 'grey', 
                               label = r'$ R( \lceil\tilde{P}_0\rceil), R( \lfloor\tilde{P}_0\rfloor)  $')
                    ax[i].plot(PV_low[option_plot_selection[i],:], linestyle = ':', color = 'grey')

                    if i%n_columns==0: # first column
                        ax[i].set_ylabel('Policy Value', fontsize = 'large')
                    if i>= (len(option_plot_selection)-n_columns): # last row
                        ax[i].set_xlabel('Time, $t$', fontsize = 'large')

                    if include_ann == True:
                        # Predicted Reserve by ANN
                        ax[i].plot(ann_prediction[option_plot_selection[i],:], 
                                                          color = 'blue', label = r'$\hat{f}_\mathcal{N}(\tilde{P}_{\mathcal{N}})$')
                        # Reserve based on classical calculation using representative contracts of ANN approach
                        ax[i].plot(ann_PV_up[option_plot_selection[i],:], color = 'orange', 
                                   label = r'$ R( \lceil\tilde{P}_\mathcal{N}\rceil), R( \lfloor\tilde{P}_\mathcal{N}\rfloor)  $')
                        ax[i].plot(ann_PV_low[option_plot_selection[i],:], color = 'orange') 

                    if i == 0:
                        ax[i].legend()
                plt.tight_layout()
            
        
        ### Look at cumulative cluster fit ###
        fig_cum, ax_cum = plt.subplots(1,1)
        ax_cum.plot((targets_cl*count).sum(axis=0), 'r*', label = '$R(P)$') # not scaled by numbers
        ax_cum.plot((PV_up*count).sum(axis=0), color = 'grey', linestyle = ':', 
                 label =r'$ R( \lceil\tilde{P}_0\rceil), R( \lfloor\tilde{P}_0\rfloor)  $')
        ax_cum.plot((PV_low*count).sum(axis=0), color = 'grey', linestyle = ':')
        if include_ann == True:
            ax_cum.plot((ann_prediction*count).sum(axis=0), label = r'$\hat{f}_\mathcal{N}(\tilde{P}_{\mathcal{N}})$')
            ax_cum.plot((ann_PV_up*count).sum(axis = 0), color = 'orange', label = r'$ R( \lceil\tilde{P}_\mathcal{N}\rceil), R( \lfloor\tilde{P}_\mathcal{N}\rfloor)  $')
            ax_cum.plot((ann_PV_low*count).sum(axis = 0), color = 'orange')
        ax_cum.set_xlabel('Time, $t$', fontsize = 'large')
        ax_cum.set_ylabel('Policy Value', fontsize = 'large')
        ax_cum.set_title('$K=$ '+str(n_cl), fontsize= 'large')
        ax_cum.legend() 
        
    elif option == 'statistic':
        # Compare target policy values with policy values of ANN representatives
        # For the PV of ANN representatives we take the mean of the floored (ann_PV_low)
        # and ceiled value (ann_PV_up) as our proxy
        
        df = pd.DataFrame(data=None, index = None, columns = ['min re${}_t$','mean re${}_t$',
                                                              'max re${}_t$',r'$D$'])
        
        ## Cumulative
        index_cum = ((targets_cl*count).sum(axis=0)>0)
        
        # Portfolio: K-Means
        diff_km = (((PV_up*count+PV_low*count)/2).sum(axis=0)-(targets_cl*count).sum(axis=0))
        diff_km_rel = diff_km[index_cum]/(targets_cl*count).sum(axis=0)[index_cum]
        vol_km_discr = ((PV_up+PV_low)/2*count).sum()/(targets_cl*count).sum()-1
        df.loc[r'$\hat{R}(\tilde{P}_0)$'] = (diff_km_rel.min(), diff_km_rel.mean(), diff_km_rel.max(), vol_km_discr)
        
        # Portfolio: ANN via classical calculation
        diff = (((ann_PV_up*count+ann_PV_low*count)/2).sum(axis=0)-(targets_cl*count).sum(axis=0))
        diff_rel = diff[index_cum]/(targets_cl*count).sum(axis=0)[index_cum]
        vol_discr = ((ann_PV_up+ann_PV_low)/2*count).sum()/(targets_cl*count).sum()-1
        df.loc[r'$\hat{R}(\tilde{P}_{\mathcal{N}})$'] = (diff_rel.min(), diff_rel.mean(), diff_rel.max(), vol_discr)
        
        # Portfolio: ANN prediction
        diff_pred = (ann_prediction*count).sum(axis=0)-(targets_cl*count).sum(axis=0)
        diff_pred_rel = diff_pred[index_cum]/(targets_cl*count).sum(axis=0)[index_cum]
        vol_pred_discr = (ann_prediction*count).sum()/(targets_cl*count).sum()-1
        df.loc[r'$\hat{f}_\mathcal{N}(\tilde{P}_{\mathcal{N}})$'] = (diff_pred_rel.min(), diff_pred_rel.mean(), 
                                                                     diff_pred_rel.max(), vol_pred_discr)
        
        if option_cluster_fit == True:
            
            fig,ax = plt.subplots(1,3, figsize = figsize)
            
            
            ##### Plot: ANN - Classical Calculation ######
            values_ann = np.zeros(shape = (n_cl, 2))
            # maximum policy value of representatives of clusters
            values_ann[:,0] = targets_cl.mean(axis=1)
            # total discrepancies (of cumulative volume) per cluster
            values_ann[:,1] = ((ann_PV_up+ann_PV_low)/2*count).sum(axis=1)/(targets_cl*count).sum(axis=1)-1
            
            
            #sort values for later rolling average mean
            values_ann = values_ann[values_ann[:,0].argsort(),:]
            
            ax[0].axhline(y=0, xmax = values_ann[:,0].max(), linestyle = '--', color = 'grey', alpha = 0.2)
            ax[0].plot(values_ann[:,0],np.cumsum(a=values_ann[:,1])/range(1,(n_cl+1)), color = 'red', 
                     linestyle = '--', label = 'Rolling Average')
            ax[0].scatter(values_ann[:,0], values_ann[:,1], label = 'Cluster')
            ax[0].set_xlabel('Mean PV of Cluster', fontsize = 'large')
            ax[0].set_ylabel(r'Discrepancy, $D_\hat{R}$', fontsize = 'large')
            ax[0].set_title(r'ANN ($\tilde{P}_\mathcal{N}$) ', fontsize = 'large')
            ax[0].legend()
            
            ######## Plot: ANN - Prediction ######
            values_ann_pred = np.zeros(shape = (n_cl, 2))
            # maximum policy value of representatives of clusters
            values_ann_pred[:,0] = targets_cl.mean(axis=1)
            # total discrepancies (of cumulative volume) per cluster
            values_ann_pred[:,1] = (ann_prediction*count).sum(axis=1)/(targets_cl*count).sum(axis=1)-1
            
            
            #sort values for later rolling average mean
            values_ann_pred = values_ann_pred[values_ann_pred[:,0].argsort(),:]
            
            ax[1].axhline(y=0, xmax = values_ann_pred[:,0].max(), linestyle = '--', color = 'grey', alpha = 0.2)
            ax[1].plot(values_ann_pred[:,0],np.cumsum(a=values_ann_pred[:,1])/range(1,(n_cl+1)), color = 'red', 
                     linestyle = '--', label = 'Rolling Average')
            ax[1].scatter(values_ann_pred[:,0], values_ann_pred[:,1], label = 'Cluster')
            ax[1].set_ylabel('Discrepancy, $D_{\hat{f}_\mathcal{N}}$', fontsize = 'large')
            ax[1].set_xlabel('Mean PV of Cluster', fontsize = 'large')
            ax[1].set_title(r'ANN ($\tilde{P}_\mathcal{N}$)', fontsize = 'large')

            
            ### Plot: K-Means ###
            values_km = np.zeros(shape = (n_cl, 2))
            # maximum policy value of representatives of clusters
            values_km[:,0] = targets_cl.mean(axis=1)
            # total discrepancies (of cumulative volume) per cluster
            values_km[:,1] = ((PV_up+PV_low)/2*count).sum(axis=1)/(targets_cl*count).sum(axis=1)-1
            
            # sort values for later time dependent mean
            values_km = values_km[values_km[:,0].argsort(),:]
            
            ax[2].axhline(y=0, xmax = values_km[:,0].max(), linestyle = '--', color = 'grey', alpha = 0.2)
            ax[2].plot(values_km[:,0],np.cumsum(a=values_km[:,1])/range(1,(n_cl+1)), color = 'red', 
                     linestyle = '--')#, label = 'Rolling Average')
            ax[2].scatter(values_km[:,0], values_km[:,1])#, label = 'Cluster')
            ax[2].set_xlabel('Mean PV of Cluster', fontsize = 'large')
            ax[2].set_ylabel(r'Discrepancy, $D_\hat{R}$', fontsize = 'large')
            ax[2].set_title(r'$K$-Means ($\tilde{P}_0$)', fontsize = 'large')
            
            #ax[2].legend()
            plt.tight_layout()
            plt.show()
             
            
            
        return (df, targets_cl, ann_PV_low,ann_PV_up, count)
    else:
        print('Unknown option!')
    
    return